In [17]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
import torch
from glasses.nn.regularization import StochasticDepth


x = torch.ones((2, 1, 1, 1))
s = StochasticDepth(0.8)
s(x)

tensor([[[[1.2500]]],


        [[[1.2500]]]])

In [21]:
import torch
from glasses.models.classification.efficientnetv2 import EfficientNetV2, EfficientNet
eff = EfficientNet().efficientnet_b3().eval().cuda()
x = torch.randn((32, 3,224, 224)).cuda()


In [22]:
%%timeit
with torch.no_grad():
    for _ in range(10):
        _ = eff(x)

617 ms ± 19.5 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [23]:
effv2 = EfficientNetV2().eval().cuda()

In [24]:
%%timeit
with torch.no_grad():

    for _ in range(10):
        _ = effv2(x)

647 ms ± 6.35 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [20]:
effv2

EfficientNetV2(
  (encoder): EfficientNetEncoder(
    (stem): ConvBnAct(
      (conv): Conv2dPad(3, 24, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (bn): BatchNorm2d(24, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (act): SiLU(inplace=True)
    )
    (layers): ModuleList(
      (0): ResNetLayer(
        (0): FusedInvertedResidualBlock(
          (block): Sequential(
            (fused): ConvBnAct(
              (conv): Conv2dPad(24, 24, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
              (bn): BatchNorm2d(24, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
              (act): SiLU(inplace=True)
            )
            (att): Identity()
            (point): Sequential(
              (0): ConvBnAct(
                (conv): Conv2dPad(24, 24, kernel_size=(1, 1), stride=(1, 1), bias=False)
                (bn): BatchNorm2d(24, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        

In [ ]:
from glasses.models.classification import AlexNet


x = torch.randn((2, 3, 224, 224))
m = AlexNet()
m.encoder.features
m(x)
[print(f.shape) for f in m.encoder.features]

In [ ]:
m.encoder.layers[1]

In [ ]:
from glasses.utils.PretrainedWeightsProvider import PretrainedWeightsProvider
from glasses.models import AutoModel
from torch import nn

class Dummy(nn.Sequential):
    def __init__(self):
        super().__init__(nn.Conv2d(3, 32, kernel_size=3), 
                        nn.Conv2d(32, 64, kernel_size=3))

dummy = Dummy()
        
provider = PretrainedWeightsProvider()
# state_dict = provider['dummy']


In [ ]:
AutoModel.models_table()

# Intepretability


In [ ]:
import requests
from PIL import Image
from io import BytesIO
from glasses.interpretability import GradCam, SaliencyMap, ScoreCam
from torchvision.transforms import Normalize
from glasses.models import AutoModel, AutoConfig

model = AutoModel.from_pretrained('resnet18').cuda()
cfg = AutoConfig.from_name('resnet18')
r = requests.get('https://i.insider.com/5df126b679d7570ad2044f3e?width=700&format=jpeg&auto=webp')
im = Image.open(BytesIO(r.content))
# un normalize when done
postprocessing = Normalize(-cfg.mean / cfg.std, (1.0 / cfg.std))
# # apply preprocessing
x =  cfg.transform(im).unsqueeze(0).cuda()
# _ = model.interpret(x, using=GradCam(), postprocessing=postprocessing).show()


ScoreCam()(x, model, postprocessing=postprocessing).show()

In [ ]:
GradCam()(x, model, postprocessing=postprocessing).show()

In [ ]:
from einops import rearrange
import torch
import torch.nn.functional as F
from

a = torch.rand((2, 10, 5, 5))

# rear = rearrange(a, 'b c h w -> b (c h w)',)

a = F.interpolate(a, size=(224, 224), mode='bilinear', align_corners=False)
# scaling
# rear.shape
mins = a.min(keepdim=True, dim=1).values
maxs = a.max(keepdim=True, dim=1).values
scaling = (a - mins) / (maxs - mins)
scaling

# Benchmark

In [ ]:
from glasses.models import AutoModel, AutoConfig
import torch

In [ ]:
model = AutoModel.from_name('vit_base_patch16_224')
model

In [ ]:
from glasses.models.classification.deit import DeiT
DeiT().embedding.tokens.dist

In [ ]:
import torch
model = torch.hub.load('facebookresearch/deit:main', 'deit_base_distilled_patch16_224', pretrained=True)

In [ ]:
model

# Clone

In [ ]:
import torch
from glasses.models import AutoModel, AutoConfig, EfficientNetLite
import timm
from transfer_weights import clone_model, deit_clone
from benchmark import benchmark
from glasses.models.classification.vit import ViTTokens

src = torch.hub.load('facebookresearch/deit:main', 'deit_small_distilled_patch16_224', pretrained=True).eval()
dst = deit_clone('deit_small_patch16_224').eval()
# src = timm.create_model('vit_base_patch16_224', pretrained='True')
# dst = AutoModel.from_name('vit_base_patch16_224')
transform = AutoConfig.from_name('vit_base_patch16_224').transform

In [ ]:
benchmark(dst.cuda(), transform, 256)

In [ ]:
benchmark(src.cuda(), transform, 256)

In [ ]:
dst.summary()